Todo:
- vodící čáry
- ticks
- osa s procenty
- různé letopočty v různých řadách

In [122]:
as_js_literal = my_chart.to_js_literal()
code = f"<html><head><title>Pokus s grafem</title></head><body>{pred}{as_js_literal}</script></body></html>"
with open(os.path.join("private","graf.html"), "w+") as f:
    f.write(code)

In [167]:
import os
import json
import pandas as pd
df = pd.read_json(os.path.join("data", "filmy.json"))

In [168]:
df = df[["Copyright","Režie","Zvuk","Scénář","Film"]].explode("Režie").explode("Zvuk").explode("Scénář")

In [188]:
rezie = pd.Series(df[df["Režie"].str.contains("g",na=False)].groupby("Copyright")["Film"].nunique().fillna(0) / df.groupby("Copyright")["Film"].nunique().fillna(0), name="Režie").fillna(0)
scenar = pd.Series(df.groupby("Copyright")["Scénář"].nunique())
zvuk = pd.Series(df.groupby("Copyright")["Zvuk"].nunique())

In [189]:
rezie

Copyright
1898    0.000000
1901    0.000000
1902    0.000000
1906    0.000000
1907    0.000000
          ...   
2018    0.090909
2019    0.106383
2020    0.057692
2021    0.081633
2022    0.000000
Name: Režie, Length: 119, dtype: float64

In [390]:
def graf(carovy=[], sloupcovy=[], procenta=[], skryte=[], max_procenta=100, target="", titulek="", podtitulek="", osay="", osay2="", kredity="", zaokrouhleni = 2, prvni=True):
    
    import os
    from highcharts_core.chart import Chart
    from highcharts_core.options.series.area import LineSeries
    from highcharts_core.options.series.bar import ColumnSeries
    from highcharts_core.options.title import Title
    from highcharts_core.options.subtitle import Subtitle
    from highcharts_core.options.credits import Credits

    if prvni:
        zdrojaky = f"""<script src="https://code.highcharts.com/highcharts.js"></script>"""
    else:
        zdrojaky = ""
    
    pred = f"""{zdrojaky}
        <figure class="target_div", id="{target}">
        <div id="container"></div>
        </figure>
        <script>"""
    
    if len(carovy) > 0:
        categories = carovy[0].index.to_list()
    else:
        categories = sloupcovy[0].index.to_list()
    categories = [str(x) for x in categories]
    
    nastaveni = {}
    nastaveni["xAxis"] = {'categories': categories, "min": 0}
    nastaveni["yAxis"] = [{'title': {'text': osay}}]
    
    if len(procenta) > 0:
        
        osa_procent = {'title' : {'text': osay2}, "max": max_procenta, "min": 0}
        
        if len(procenta) != len(carovy) + len(sloupcovy):
            osa_procent["opposite"] = True
            druha_osa = 1
            nastaveni["yAxis"].append(osa_procent)
            nastaveni["alignTicks"] = False
        if len(procenta) == len(carovy) + len(sloupcovy):
            nastaveni["yAxis"] = [osa_procent]
            druha_osa = 0
 
    my_chart = Chart(container = target, options = nastaveni)
    
    procenta = [p.name for p in procenta]
    skryte = [s.name for s in skryte]
    
    def vykresleni(serie, typ):
        for s in serie:
            popisek = s.name
            if s.name in skryte:
                viditelnost = False
            else:
                viditelnost = True
            if s.name in procenta:
                s = [round(x * 100, zaokrouhleni) for x in s.fillna(0).to_list()]
                my_chart.add_series(typ(data = s, visible=viditelnost, name=popisek, y_axis = druha_osa, tooltip={"valueSuffix": " %"}))
            else:
                my_chart.add_series(typ(data = s.fillna(0).to_list(), visible=viditelnost, name=popisek, y_axis = 0))
    
    if len(sloupcovy) > 0:
        vykresleni(sloupcovy, ColumnSeries)
    if len(carovy) > 0:
        vykresleni(carovy, LineSeries)
    
    my_chart.options.title = Title(text = titulek)
    
    if len(podtitulek) > 0:
        my_chart.options.subtitle = Subtitle(text = podtitulek)

    my_credits = Credits(text = kredity[0], enabled = True, href = kredity[1])
    my_chart.options.credits = my_credits

    as_js_literal = my_chart.to_js_literal()
        
    code = f"<html><head><title>Pokus s grafem</title></head><body>{pred}{as_js_literal}</script></body></html>"

    if not os.path.exists("grafy"):
        os.mkdir("grafy")
    
    filename = target + ".html"
    
    with open(os.path.join("grafy",filename), "w+") as f:
        f.write(code)
        print("Graf uložen.")

In [388]:
graf(carovy=[rezie, scenar], sloupcovy=[zvuk], procenta=[rezie], target="filmiky", titulek="Něco", podtitulek="Něco jiného", kredity=["Zdroj dat: Filmový přehled | Vizualizace: Michal Kašpárek, iROZHLAS.cz","https://www.irozhlas.cz/zpravy-tag/datova-zurnalistika"], osay="Počet filmů")

Graf uložen.
